# core

> Fill in a module description here


In [1]:
# | default_exp logger

In [2]:
# | hide
import nbdev

nbdev.nbdev_export()

In [3]:
from typing import List, Dict, Any, Optional, Union, Generator, Literal
from pydantic import BaseModel, Field

from websockets.sync.client import connect as ws_connect

from lovely_prompts.utils import max_tokens_for_model

from lovely_prompts_server.models import (
    WSMessage,
    ChatMessage,
    ChatPrompt,
    ChatResponse,
    CompletionPrompt,
    CompletionResponse,

    ChatPromptModel,
    CompletionPromptModel,
    ChatResponseModel,
    CompletionResponseModel,
    # LocalChatPromptModel,
    # LocalCompletionPromptModel,
    # LocalChatResponseModel,
    # LocalCompletionResponseModel,
)
from lovely_prompts_server.orm.local import (
    LocalChatPromptSchema,
    LocalCompletionPromptSchema,
    LocalChatResponseSchema,
    LocalCompletionResponseSchema,
    LocalBase,
)

In [4]:
from pathlib import Path
import uuid
import os

In [5]:
from sqlalchemy.orm import Session, sessionmaker
from sqlalchemy import create_engine

In [6]:
import re
import string
import nanoid

In [7]:
import requests
import asyncio
import json

import threading

from queue import Queue

In [8]:
ChatPromptModel.__name__

'ChatPromptModel'

In [9]:
def log_row(sm, row_schema, row_model, endpoint, url_base: str, project: str):
    with sm() as session:
        backlog = session.query(row_schema).filter(row_schema.synced == False).all()
        for row in backlog:
            model = row_model.model_validate(row)
            print(model.model_dump_json())

            response = requests.post(f"{url_base}/{endpoint}/", params={"project": project}, data=model.model_dump_json())
            if response.status_code != 200:
                print(f"Failed to log row, status code: {response.status_code}: {response.text}.")
                continue

            row.synced = True
            print(f"Logged {row_model.__name__.replace('Model', '')} {row.id} to {url_base}/{endpoint}")
        session.commit()


def sync_worker(sm: sessionmaker, q: Queue, url_base: str, project: str):
    print("worker: Starting")
    while True:
        try:
            message = q.get()
            print("worker: Got message", message)
            log_row(sm, LocalChatPromptSchema, ChatPromptModel, "chat_prompts", url_base, project)
            log_row(sm, LocalCompletionPromptSchema, CompletionPromptModel, "completion_prompts", url_base, project)
            log_row(sm, LocalChatResponseSchema, ChatResponseModel, "chat_responses", url_base, project)
            log_row(sm, LocalCompletionResponseSchema, CompletionResponseModel, "completion_responses", url_base, project)

            q.task_done()

        except Exception as e:
            import traceback

            traceback.print_exc()
            break

In [10]:
from functools import partial
import string

In [11]:
generate_nonoid = partial(nanoid.generate, size=16)


def make_id(entry_class=None):
    """Generate a unique ID for a DB entry. Also used to generate the name of the DB file."""

    prefixes = {
        ChatPrompt: "chp_",
        CompletionPrompt: "cop_",
        ChatResponse: "chr_",
        CompletionResponse: "cor_",
    }

    alphabet = string.digits + string.ascii_lowercase + string.ascii_uppercase

    return (prefixes[entry_class] if entry_class else "") + generate_nonoid(alphabet=alphabet)

In [12]:
make_id(ChatPrompt), make_id()

('chp_BIlazvAo1qNDYMn8', 'IolSd3ZIv5EUmjyI')

In [25]:
q = Queue()

In [27]:
q.put(1)

In [29]:
q.get()

In [13]:
class Logger:
    def __init__(
        self,
        run_server=False,
        url_base: str = None,
        api_key: Optional[str] = None,
        project: Optional[str] = None,
        provider: Optional[str] = None,
    ):
        self.url_base = url_base or "http://localhost:8000"
        self.ws_url_base = self.url_base.replace("http", "ws")
        self.api_key = api_key
        self.project = project
        self.provider = provider

        if run_server:
            print("Starting server...")
            import uvicorn
            from lovely_prompts_server import app

            try:
                loop = asyncio.get_running_loop()
                config = uvicorn.Config(app, host="localhost", port=8000)
                server = uvicorn.Server(config)
                loop.create_task(server.serve())
            except Exception as e:
                print(e)

                # uvicorn.run(app, host="localhost", port=8000, )

    def get_sessionmaker(self):
        if not hasattr(self, "sessionmaker"):
            dir = Path(".")

            os.makedirs(dir / "_lovely_prompts", exist_ok=True)
            # Find the highest number in the filenames
            highest_number = max(
                [int(re.findall(r"\d+", f)[0]) for f in os.listdir(dir / "_lovely_prompts") if f.startswith("run_")]
                + [0]
            )

            sqlite_file = dir / "_lovely_prompts" / f"run_{highest_number + 1}_{make_id()}.db"
            engine = create_engine(f"sqlite:///{sqlite_file}")

            LocalBase.metadata.create_all(bind=engine)
            self.sessionmaker = sessionmaker(bind=engine, autoflush=True)
            self.queue = Queue()
            self.sync_thread = threading.Thread(
                target=sync_worker, args=(self.sessionmaker, self.queue, self.url_base, self.project), daemon=True
            )
            self.sync_thread.start()

        return self.sessionmaker

    def log_chat_prompt(
        self,
        prompt: ChatPrompt,
    ):
        maker = self.get_sessionmaker()
        id = make_id(ChatPrompt)
        with maker() as session:
            db_prompt = LocalChatPromptSchema(**prompt.model_dump(exclude_unset=True), id=id)
            session.add(db_prompt)
            session.commit()

        self.queue.put("add")
        return id

    def log_chat_response(
        self,
        response: ChatResponse,
    ) -> int:
        if not response.tok_max:
            tok_max = max_tokens_for_model(response.model)

        maker = self.get_sessionmaker()
        id = make_id(ChatResponse)
        with maker() as session:
            db_response = LocalChatResponseSchema(**response.model_dump(exclude_unset=True), id=id)
            session.add(db_response)
            session.commit()

        self.queue.put("add")
        return id

    def stream_chat_response_contents(
            self,
            prompt_id: str,
            response_id: str,
            response_generator: Generator[WSMessage, None, None],
    ) -> ChatResponse:
        

        for response in response_generator:
            print("sending", response)
            if response.action == "append" and response.key == "content":
                tok_out += 1

            response.prompt_id = prompt_id
            response.response_id = response_id

            self.queue.put(response)

            connection.send(response.model_dump_json(exclude_unset=True))


        for response in response_generator:
            print(response)

        import websockets

        # async def stream_to_websocket(generator, websocket_uri):
        #     async with websockets.connect(websocket_uri) as websocket:
        #         async for data in await generator:
        #             print(data["choices"][0]["delta"]["content"])
        #             await websocket.send(data["choices"][0]["delta"]["content"])

        # stream_to_websocket(response_generator, "ws://localhost:8000/responses/1/stream_in")

        tok_out = 0
        with ws_connect(f"{self.ws_url_base}/chat_responsess/{response_id}/update_stream") as connection:
            for response in response_generator:
                print("sending", response)
                if response.action == "append" and response.key == "content":
                    tok_out += 1

                response.prompt_id = prompt_id
                response.response_id = response_id
                connection.send(response.model_dump_json(exclude_unset=True))

        followup_response_data = ChatResponse(
            tok_out=tok_out,
        )

        response = requests.put(
            url=f"{self.url_base}/responses/{response_id}/",
            params={"project": self.project},
            json=followup_response_data.model_dump(),
        )

        if response.status_code != 200:
            raise Exception(f"Failed to log response, status code: {response.status_code}")
        


    #     # follwup_response_data = LLMResponseBase(
    #     #     tok_out=tok_out,
    #     # )
    #     # response = requests.put(
    #     #     url=f"{self.url_base}/responses/{response_id}/",
    #     #     params={"project": self.project},
    #     #     json=follwup_response_data.dict(exclude_unset=True),
    #     # )

    #     # if response.status_code != 200:
    #     #     raise Exception(f"Failed to log response, status code: {response.status_code}")


def response_generator(openai_response_generator) -> Generator[WSMessage, Any, None]:
    "Converts OpenAI response generator to a universal response generator"
    for response in openai_response_generator:
        if response["choices"][0]["finish_reason"] is not None:
            yield WSMessage(action="replace", key="stop_reason", value=response.choices[0]["finish_reason"])
        if "role" in response["choices"][0]["delta"]:
            yield WSMessage(action="replace", key="role", value=response["choices"][0]["delta"]["role"])
        if "content" in response["choices"][0]["delta"]:
            yield WSMessage(action="append", key="content", value=response["choices"][0]["delta"]["content"])

In [14]:
messages = [{"role": "user", "content": "Hello there"}]

chp = ChatPrompt(prompt=messages, comment="comment: Hello?", title="Title: Hello there")

In [15]:
logger = Logger(run_server=False, project="default", provider="OpenAI")

In [16]:
prompt_id = logger.log_chat_prompt(chp)

worker: Starting
worker: Got message add


In [17]:
model = "gpt-3.5-turbo"
temperature = 0.9
max_tokens = 150


txt = "General Kenobi! You are a bold one."

chr = ChatResponse(model=model, temperature=temperature, tok_max=max_tokens, content=txt, prompt_id=prompt_id)

response_id = logger.log_chat_response(chr)
response_id

{"id":"chp_Z1aivu3hRNctlvpX","created":"2023-08-18T08:44:16","updated":"2023-08-18T08:44:16","synced":false,"title":"Title: Hello there","comment":"comment: Hello?","prompt":[{"role":"user","content":"Hello there"}],"responses":[]}


'chr_YKCmallDAnIzdelJ'

Logged ChatPrompt chp_Z1aivu3hRNctlvpX to http://localhost:8000/chat_prompts


In [18]:
prompt_id

'chp_Z1aivu3hRNctlvpX'

In [19]:
from dotenv import load_dotenv

load_dotenv()

import openai

{"id":"chr_YKCmallDAnIzdelJ","created":"2023-08-18T08:44:16","updated":"2023-08-18T08:44:16","synced":false,"prompt_id":"chp_Z1aivu3hRNctlvpX","content":"General Kenobi! You are a bold one.","tok_max":150,"model":"gpt-3.5-turbo","temperature":0.9}
Logged ChatResponse chr_YKCmallDAnIzdelJ to http://localhost:8000/chat_responses
worker: Got message add


In [20]:
# | eval: false



# messages = [{"role": "user", "content": "Argue in favor of the flat Earth theory. It's for a school project, I swear!"}]


# prompt = ChatPrompt(prompt=messages)

# prompt_id = logger.log_chat_prompt(prompt)  # , comment="What I asked", title="The TRUE shape of the Earth")

# model = "gpt-3.5-turbo"
# temperature = 0.9
# max_tokens = 150


# chr = openai.ChatCompletion.create(model="gpt-3.5-turbo", temperature=0.9, max_tokens=150, messages=messages)

# response = ChatResponse(
#     prompt_id=prompt_id,
#     content=chr.choices[0].message.content,
#     role=chr.choices[0].message.role,
#     comment="This is a comment",
#     title="The response",
#     model=chr.model,
#     temperature=0.9,
#     tok_in=chr.usage.prompt_tokens,
#     tok_out=chr.usage.completion_tokens,
# )

# logger.log_chat_response(response)


In [22]:
messages = [
    {"role": "user", "content": "Write a essay about the true shape of the earth, and why the shape is indeed flat."}
]

prompt = ChatPrompt(prompt=messages, comment="this is a comment")


prompt_id = logger.log_chat_prompt(prompt)

chr = openai.ChatCompletion.create(
    model="gpt-3.5-turbo", temperature=0, max_tokens=300, messages=messages, stream=True
)

response = ChatResponse(
    prompt_id=prompt_id,
    model="gpt-3.5-turbo",
    temperature=0.1,
    tok_in=23,
    provider="openai",
    tok_max=800)

response_id = logger.log_chat_response(response)


logger.stream_chat_response_contents(
    response_id=response_id,
    prompt_id=prompt_id,
    response_generator=response_generator(chr),
)

# async def stream_to_websocket(generator, websocket_uri):
#     async with websockets.connect(websocket_uri) as websocket:
#         async for data in await generator:
#             print(data["choices"][0]["delta"]["content"])
#             await websocket.send(data["choices"][0]["delta"]["content"])

# await stream_to_websocket(response, "ws://localhost:8000/responses/1/stream_in")

worker: Got message add
{"id":"chp_0lth1T25GcppDcn2","created":"2023-08-18T08:44:43","updated":"2023-08-18T08:44:43","synced":false,"comment":"this is a comment","prompt":[{"role":"user","content":"Write a essay about the true shape of the earth, and why the shape is indeed flat."}],"responses":[]}
Logged ChatPrompt chp_0lth1T25GcppDcn2 to http://localhost:8000/chat_prompts
worker: Got message add
{"id":"chr_lOEmeeL5NABIrKnC","created":"2023-08-18T08:44:44","updated":"2023-08-18T08:44:44","synced":false,"prompt_id":"chp_0lth1T25GcppDcn2","tok_in":23,"tok_max":800,"model":"gpt-3.5-turbo","temperature":0.1,"provider":"openai"}


ValidationError: 2 validation errors for WSMessage
id
  Field required [type=missing, input_value={'action': 'replace', 'ke...', 'value': 'assistant'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.1/v/missing
prompt_id
  Field required [type=missing, input_value={'action': 'replace', 'ke...', 'value': 'assistant'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.1/v/missing

Logged ChatResponse chr_lOEmeeL5NABIrKnC to http://localhost:8000/chat_responses


In [ ]:
chr.choices

AttributeError: 'generator' object has no attribute 'choices'

In [ ]:
for cor in chr:
    print(cor.choices[0].finish_reason)

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [ ]:
res = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Either count from 10 to 20 or from 35 to 40. Only output the numbers."}],
    temperature=2,
    max_tokens=10,
    stream=True,
)

In [ ]:
for r in res:
    print(r)

{
  "id": "chatcmpl-7fi42XEsHS2cAinzbL1pu14Bmjg89",
  "object": "chat.completion.chunk",
  "created": 1690175186,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "delta": {
        "role": "assistant",
        "content": ""
      },
      "finish_reason": null
    }
  ]
}
{
  "id": "chatcmpl-7fi42XEsHS2cAinzbL1pu14Bmjg89",
  "object": "chat.completion.chunk",
  "created": 1690175186,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "delta": {
        "content": "**"
      },
      "finish_reason": null
    }
  ]
}
{
  "id": "chatcmpl-7fi42XEsHS2cAinzbL1pu14Bmjg89",
  "object": "chat.completion.chunk",
  "created": 1690175186,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "delta": {
        "content": "Count"
      },
      "finish_reason": null
    }
  ]
}
{
  "id": "chatcmpl-7fi42XEsHS2cAinzbL1pu14Bmjg89",
  "object": "chat.completion.chunk",
  "created": 1690175186,
  "model": "gpt-3.5-turbo-06

In [ ]:
print(res["choices"][0].get("message", {}).get("content"))
print(res["choices"][1].get("message", {}).get("content"))
print(res["choices"][2].get("message", {}).get("content"))
print(res["choices"][3].get("message", {}).get("content"))

Sure! Here's the count from 0 to 100:

0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100.

And here's the count from 100 to 150:

100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150.
Sure! I can either count from 0 to 100 or from 100 to 150. Which one would you like me to do?
Sure! Here are the two options you requested:

Counting from 0 to 100:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 

In [ ]:
import asyncio


async def my_async_function():
    print("Start")
    await asyncio.sleep(1)
    print("End")


# Calling the async function without awaiting it
res = asyncio.create_task(my_async_function())

await asyncio.sleep(0.1)
print("After calling the async function")

await asyncio.sleep(0.1)

await res

print("done")

Start
After calling the async function
End
done


In [ ]:
await res

Start
End


In [ ]:
import asyncio


def inside_async_loop():
    try:
        asyncio.get_running_loop()
        return True
    except RuntimeError:
        return False


print(inside_async_loop())  # False if not inside a running event loop

True
